# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298668674651                   -0.85    5.0         
  2   -8.300219972802       -2.81       -1.26    1.0    771ms
  3   -8.300437201046       -3.66       -1.89    3.1    140ms
  4   -8.300461040666       -4.62       -2.75    2.0    125ms
  5   -8.300463846779       -5.55       -3.04    3.0    156ms
  6   -8.300464246983       -6.40       -3.25    1.4    142ms
  7   -8.300464441006       -6.71       -3.39   11.8    255ms
  8   -8.300464553135       -6.95       -3.54    1.9    129ms
  9   -8.300464616185       -7.20       -3.73    1.6    123ms
 10   -8.300464634981       -7.73       -3.91    4.8    179ms
 11   -8.300464643627       -8.06       -4.27    2.9    194ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67418065897                   -0.70    6.8         
  2   -16.67866279907       -2.35       -1.14    1.0    277ms
  3   -16.67919078964       -3.28       -1.87    2.9    332ms
  4   -16.67927651949       -4.07       -2.75    3.4    344ms
  5   -16.67928596600       -5.02       -3.17    6.1    445ms
  6   -16.67928618530       -6.66       -3.46    2.5    318ms
  7   -16.67928621384       -7.54       -3.90    2.2    295ms
  8   -16.67928622130       -8.13       -4.52    2.8    322ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32583688804                   -0.56    7.5         
  2   -33.33272767613       -2.16       -1.00    1.0    977ms
  3   -33.33405671110       -2.88       -1.73    5.1    1.32s
  4   -33.33428250225       -3.65       -2.63    2.2    1.08s
  5   -33.33694150771       -2.58       -2.49    7.0    1.60s
  6   -33.33694163240       -6.90       -2.46    6.5    1.30s
  7   -33.33694293565       -5.88       -3.14    1.1    911ms
  8   -33.33694356571       -6.20       -3.49    5.2    1.24s
  9   -33.33694377490       -6.68       -4.06    2.5    1.09s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298294150724                   -0.85    5.1         
  2   -8.300249937471       -2.71       -1.59    1.0   86.6ms
  3   -8.300439010180       -3.72       -2.64    3.2    127ms
  4   -8.300424601963   +   -4.84       -2.45    7.9    219ms
  5   -8.300464252834       -4.40       -3.41    1.0    121ms
  6   -8.300464567564       -6.50       -3.80    3.2    141ms
  7   -8.300464641369       -7.13       -4.43    1.4   98.8ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32628976668                   -0.56    6.5         
  2   -33.32237561910   +   -2.41       -1.27    1.2    948ms
  3   -19.19846237786   +    1.15       -0.49    6.4    2.24s
  4   -33.31826042481        1.15       -1.81    5.8    1.71s
  5   -33.02798051254   +   -0.54       -1.22    3.8    1.50s
  6   -32.29980173423   +   -0.14       -1.05    4.4    1.47s
  7   -33.33395827524        0.01       -2.07    4.2    1.41s
  8   -33.33417070127       -3.67       -2.24    2.5    1.09s
  9   -33.33621702860       -2.69       -2.43    1.9    1.05s
 10   -33.33641646569       -3.70       -2.66    1.5    947ms
 11   -33.33682512095       -3.39       -2.96    2.4    1.05s
 12   -33.33692938489       -3.98      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.